<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class3-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 3.1**
Week 3!  You are 20% done with the class.

**Announcements**
* Watch [Video No. 5: Quantization](https://youtu.be/T2i5ddP4dFQ) for next class
* [Music Tech Link event](https://bit.ly/phl-mtl) tomorrow (Tue 10/1) at 5pm (ExCITe Center: 3401 Market St.)
* Lectures are in-person only, for now.





# Optional stuff

Install `ipympl` for  Matplotlib


In [ ]:
!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib widget

My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
As always, start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class3.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'thx1138'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

## `myPlot()` function
A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt=''):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  fig.tight_layout()
  plt.ion()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Sinusoids, revisited

## Complex exponential phasor with $\cos()$ and $\sin()$ lines

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize = (8,8))

ax = plt.axes(xlim=(-2.2, 2.2), ylim=(-2.2, 2.2))
plt.close()   # Don't output the final figure separately

N = 100
c_n = np.arange(N+1)
c_x = np.cos(2*np.pi*c_n/N)
c_y = np.sin(2*np.pi*c_n/N)

fig.tight_layout()

f = 1/30 / 2
circle, = ax.plot(c_x,c_y,'lightgray') 
cos_guide, = ax.plot([], [], 'g:')
sin_guide, = ax.plot([], [], 'r:')
dot_ccw, = ax.plot([], [], 'yo', markersize=15)

# animation function.  This is called sequentially
def animate(n):
    x0 = np.cos(2*np.pi * n*f)
    y0 = np.sin(2*np.pi * n*f)
    dot_ccw.set_data(x0, y0)
    cos_guide.set_data([0, x0], [0, 0])
    sin_guide.set_data([x0, x0], [0, y0])
    return dot_ccw, 

# call the animator.  blit=True means only re-draw the parts that have changed.=
anim = animation.FuncAnimation(fig, animate, frames=60, interval=1000/30, blit=True)

anim

##Two phasors on unit circle with sum

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize = (8,8))
ax = plt.axes(xlim=(-2.2, 2.2), ylim=(-2.2, 2.2))
ax.set_xlabel('Real')
ax.set_ylabel('Imag')
plt.close()   # Don't output the final figure separately

# Compute the x,y coordinates of the unit circle
N = 100   # 100 points is plenty
c_n = np.arange(N+1)
c_x = np.cos(2*np.pi*c_n/N)
c_y = np.sin(2*np.pi*c_n/N)

fig.tight_layout()

# Setup the plot objects that will update frame-to-frame
f = 1 / 30 / 2   # A frequency of 1/2 Hz (2 seconds around the circle, 30 frames per second)
circle, = ax.plot(c_x,c_y,'lightgray') 
dot_ccw, = ax.plot([], [], 'yo', markersize=15)
dot_cw, = ax.plot([], [], 'co', markersize=15)
dot_sum, = ax.plot([], [], 'go', markersize=15)

# Function to compute next animation frame.  This is called sequentially.
def cos_frame(n):
    x0 = np.cos(2*np.pi * n*f)
    y0 = np.sin(2*np.pi * n*f)
    dot_ccw.set_data(x0, y0)
    dot_cw.set_data(x0, -y0)
    dot_sum.set_data(2*x0, 0)

    return dot_ccw, #dot_cw, dot_sum, #line, head, guide

cos_anim = animation.FuncAnimation(fig, cos_frame, frames=60, interval=1000/30, blit=True)

cos_anim  # Display the animated frames

##Two phasors on unit circle for $\sin()$

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize = (8,8))

ax = plt.axes(xlim=(-2.2, 2.2), ylim=(-2.2, 2.2))
plt.close()   # Don't output the final figure separately

N = 100
c_n = np.arange(N+1)
c_x = np.cos(2*np.pi*c_n/N)
c_y = np.sin(2*np.pi*c_n/N)

fig.tight_layout()

f = 1/30 / 2
circle, = ax.plot(c_x,c_y,'lightgray') 
dot_ccw, = ax.plot([], [], 'yo', markersize=15)
dot_cw, = ax.plot([], [], 'mo', markersize=15)
dot_sum, = ax.plot([], [], 'ro', markersize=15)

# animation function.  This is called sequentially
def animate(n):
    x0 = np.cos(2*np.pi * n*f)
    y0 = np.sin(2*np.pi * n*f)
    dot_ccw.set_data(x0, y0)
    dot_cw.set_data(-x0, y0)
    dot_sum.set_data(0, 2*y0)

    return dot_ccw, #dot_cw, dot_sum, #line, head, guide

# call the animator.  blit=True means only re-draw the parts that have changed.=
anim = animation.FuncAnimation(fig, animate, #init_func=init,
                               frames=60, interval=1000/30, blit=True)

# Note: below is the part which makes it work on Colab
#rc('animation', html='jshtml')
anim

# Load a new audio example
Yo Yo Ma: Bach Cello Suite No. 1
* 44.1 kHz sampling rate

In [ ]:
yy44,fs44 = sf.read(path + 'YoYoMa-44kHz.wav')
ipd.Audio(yy44,rate=fs44)

In [ ]:
myPlot(yy44)

# The `fft` and `fftshift` functions

In [ ]:
N = 2048
h_win = np.hanning(N)

YY = np.fft.fft(yy44[:N] * h_win)
YY_mag = np.abs(YY)
YY_norm = 4 * YY_mag / N    # Frequency magnitude, normalized by length. Why 4?
                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                            #    x2 because hanning window has 0.5 average

fig = plt.figure(figsize=(16,6))
f = np.arange(N) * fs44 / N
plt.plot(f, 20*np.log10(np.abs(YY_norm)))
plt.xlabel('Frequency (Hz)')

In [ ]:
YY_shift = np.fft.fftshift(YY)
YY_shift_norm = 4 * np.abs(YY_shift) / N
f_shift = (np.arange(N) - N/2) * fs44 / N

fig = plt.figure(figsize=(16,6))
plt.plot(f_shift, 20*np.log10(np.abs(YY_shift_norm)))
plt.xlabel('Frequency (Hz)')

## myPlotFFT
Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = 
  
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_fft     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

In [ ]:
myPlotFFT(yy44[:44100], win='rect', neg_f=True)

# STFT: Spectrogram

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy44, fs44, window='hann', nperseg=2048, noverlap=2048-512, nfft=4096)

fig = plt.figure(figsize=(16,6))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)+1e-15))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')


# Load low sampling rate version (11025 Hz sampling)

In [ ]:
yy11,fs11 = sf.read(path + 'YoYoMa-11kHz.wav')
ipd.Audio(yy11,rate=fs11)

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy11, fs11, window='hann', nperseg=512, noverlap=256, nfft=512)

fig = plt.figure(figsize=(16,6))
#ax = plt.axes(xlim=(0,13),ylim=(0,5000))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

plt.ylim(0,15000)

#fig.savefig(path + username + '-Spectrogram-YY.png', dpi=200)

#Sample rate conversion: done poorly


## Upsampling: insert zeros between existing samples

In [ ]:
L = 4
yy11_44u = np.zeros( len(yy11)*L - (L-1) )
yy11_44u[::4] = yy11                # Put our 11 kHz wave every 4th sample

ipd.Audio(yy11_44u,rate=fs44)


In [ ]:
myPlot(yy11_44u[:200])

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy11_44c, fs44, window='hann', nperseg=512, noverlap=256, nfft=512)

fig = plt.figure(figsize=(16,6))
#ax = plt.axes(xlim=(0,13),ylim=(0,5000))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

#plt.xlim(0,4)
# plt.ylim(0,15000)

#fig.savefig(path + username + '-Spectrogram-YY.png', dpi=200)
plt.show()

# Moving Average

In [ ]:
L = 5  # Number of samples to average

yy11_44_ma = np.zeros(len(yy11_44u))               # Create an output array of zeros

for n in range(len(yy11_44u)-L):
  yy11_44_ma[n] = np.sum(yy11_44u[n:n+L]) / L   # Compute moving average of L samples

In [ ]:
myPlot(yy11_44_ma[:200])
ipd.Audio(y11_44_ma, rate=44100)

## Look at the moving average in the frequency domain

In [ ]:
mv_avg = np.ones(L) / L

myPlotFFT(mv_avg, n_fft=2048, win='rect')

In [ ]:
f1, t1, Sxx = signal.spectrogram(yy11_44_ma, fs44, window='hann', nperseg=512, noverlap=256, nfft=512)

fig = plt.figure(figsize=(16,6))

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

#fig.savefig(path + username + '-Spectrogram-YY_ma.png', dpi=200)
plt.show()

## mySpectrogram
A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 
  S_dBFS = 
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

In [ ]:
mySpectrogram(yy11_44_ma)